# 데이터 생성

In [1]:
import numpy as np
from scipy.stats import poisson

class IntentionRegionMissileMapping:
    def __init__(self):
        self.mapping = {}

    def add_intention_pair(self, intention1, intention2, regions, missiles):
        intention_pair = (intention1, intention2)
        self.mapping[intention_pair] = {
            'regions': set(regions),
            'missiles': set(missiles)
        }

    def get_regions_missiles(self, intention1, intention2):
        intention_pair = (intention1, intention2)
        return self.mapping.get(intention_pair, None)

    def get_all_mappings(self):
        return self.mapping

class MissileScenarioGenerator:
    def __init__(self, missile_types, targets, time_steps, active_regions):
        self.missile_types = missile_types
        self.targets = targets
        self.time_steps = time_steps
        self.transition_matrix = None
        self.poisson_param_model = None
        self.active_regions = active_regions

    def set_transition_matrix(self, matrix):
        self.transition_matrix = np.array(matrix)

    def set_poisson_param_model(self, model_type, params):
        self.poisson_param_model = (model_type, params)

    def generate_poisson_params(self):
        model_type, params = self.poisson_param_model
        if model_type == "linear":
            a, b = params
            return np.linspace(a, b, self.time_steps)
        elif model_type == "quadratic":
            a, b, c = params
            x = np.linspace(0, 1, self.time_steps)
            return a * x**2 + b * x + c
        elif model_type == "exponential":
            a, b = params
            return a * np.exp(b * np.linspace(0, 1, self.time_steps))
        else:
            raise ValueError("Unsupported model type")
    def select_initial_state(self) -> int:
        num_states = len(self.targets) * len(self.missile_types)
        weights = np.ones(num_states)

        for i, target in enumerate(self.targets):
            if target in self.active_regions:
                weights[i * len(self.missile_types): (i + 1) * len(self.missile_types)] *= 5  # 가중치 factor

        weights /= weights.sum()  # 정규화
        return np.random.choice(num_states, p=weights)

    def generate_scenario(self):
        num_states = len(self.targets) * len(self.missile_types)
        current_state = np.random.choice(num_states)

        current_state = self.select_initial_state()

        scenario = []
        poisson_params = self.generate_poisson_params()

        for t in range(self.time_steps):
            target_idx = current_state // len(self.missile_types)
            missile_idx = current_state % len(self.missile_types)

            target = self.targets[target_idx]
            missile_type = self.missile_types[missile_idx]

            # 포아송 분포를 사용하여 미사일 수 결정
            num_missiles = np.random.poisson(poisson_params[t])

            scenario.append((target, missile_type, num_missiles))

            # 다음 상태로 전이
            current_state = np.random.choice(num_states, p=self.transition_matrix[current_state])

        return scenario

class ScenarioDataGenerator:
    def __init__(self, intention_mapping, all_regions, all_missiles):
        self.intention_mapping = intention_mapping
        self.all_regions = all_regions
        self.all_missiles = all_missiles
        self.generator = None

    def generate_random_transition_matrix(self, active_regions, active_missiles,k1=5,k2=3):
        num_regions = len(self.all_regions)
        num_missiles = len(self.all_missiles)
        size = num_regions * num_missiles
        matrix = np.random.rand(size, size)
        
        # 가중치 적용 단계
        for i in range(size):
            region_i = i // num_missiles
            for j in range(size):
                region_j = j // num_missiles
                if self.all_regions[region_j] in active_regions:
                    #print("region",i,j)
                    matrix[i][j] *= k1
                else:
                    matrix[i][j] *= 1
        pullup_index = self.all_missiles.index("단거리 풀업")
        for i in range(size):
            region_i = i // num_missiles
            for j in range(size):
                region_j = j // num_missiles
                missile_j = j % num_missiles
                if self.all_regions[region_j] in active_regions and missile_j == pullup_index:
                    matrix[i][j] *= k2

        # 행 별로 정규화
        matrix /= matrix.sum(axis=1, keepdims=True)
        return matrix

    def generate_data(self, intention1, intention2, time_steps, transition_matrix=None, poisson_param_model=None):
        intention_pair = (intention1, intention2)
        regions_missiles = self.intention_mapping.get_regions_missiles(intention1, intention2)
        
        if not regions_missiles:
            raise ValueError(f"No mapping found for intention pair: {intention_pair}")
        
        active_regions = list(regions_missiles['regions'])
        active_missiles = list(regions_missiles['missiles'])
        
        # transition_matrix가 제공되지 않은 경우 랜덤하게 생성
        if transition_matrix is None:
            transition_matrix = self.generate_random_transition_matrix(active_regions, active_missiles)
        
        # poisson_param_model이 제공되지 않은 경우 기본값 설정
        if poisson_param_model is None:
            poisson_param_model = ("linear", (2, 8))
        
        # MissileScenarioGenerator 초기화 및 설정
        self.generator = MissileScenarioGenerator(self.all_missiles, self.all_regions, time_steps, active_regions)
        self.generator.set_transition_matrix(transition_matrix)
        self.generator.set_poisson_param_model(*poisson_param_model)
        
        # 시나리오 생성
        scenario = self.generator.generate_scenario()
        
        # 결과를 저장할 numpy 배열 초기화
        result = np.zeros((time_steps, len(self.all_regions), len(self.all_missiles)), dtype=int)
        
        # 시나리오 데이터를 numpy 배열로 변환
        for t, (target, missile, num) in enumerate(scenario):
            region_idx = self.all_regions.index(target)
            missile_idx = self.all_missiles.index(missile)
            result[t, region_idx, missile_idx] = num
        
        return result, transition_matrix, self.generator.generate_poisson_params()


def analyze_transition_matrix(transition_matrix, all_regions, all_missiles, active_regions):
    """
    전이행렬을 분석하여 active_region, 비active_region, 단거리 풀업, 비단거리 풀업에 대한 종합적인 결과를 출력합니다.
    
    :param transition_matrix: 2D numpy array, 전이행렬
    :param all_regions: list, 모든 지역 목록
    :param all_missiles: list, 모든 미사일 목록
    :param active_regions: list, 활성화된 지역 목록
    """
    num_regions = len(all_regions)
    num_missiles = len(all_missiles)
    
    # 지역 분석
    active_indices = [all_regions.index(region) for region in active_regions]
    active_prob = np.sum(transition_matrix[:, [i * num_missiles + j for i in active_indices for j in range(num_missiles)]])
    non_active_prob = np.sum(transition_matrix) - active_prob
    
    num_active = len(active_indices)
    num_non_active = len(all_regions)-len(active_indices)
    
    print("지역 분석:")
    print(f"Active 지역 평균 공격 지표: {active_prob/num_active:.4f} ")
    print(f"Non-active 지역 평균 공격 지표: {non_active_prob/num_non_active:.4f}")
    if non_active_prob > 0:
        print(f"Active/Non-active 비율: {active_prob*num_non_active/non_active_prob/num_active:.2f}")
        print(f"Active 지역이 Non-active 지역보다 {active_prob*num_non_active/non_active_prob/num_active:.2f}배 더 자주 공격 대상이 될 것으로 예상됩니다.")
    else:
        print("Non-active 지역의 확률이 0이므로 비율을 계산할 수 없습니다.")
    
    # 미사일 분석
    pullup_index = all_missiles.index("단거리 풀업")
    pullup_prob = np.sum(transition_matrix[:, [i * num_missiles + pullup_index for i in active_indices]])
    non_pullup_prob = active_prob - pullup_prob
    
    print("\n미사일 분석:")
    print(f"단거리 풀업 발사 지표: {pullup_prob:.4f}")
    print(f"비단거리 풀업 발사 지표: {non_pullup_prob/4:.4f} ")
    if non_pullup_prob > 0:
        print(f"단거리 풀업/비단거리 풀업 비율: {pullup_prob*4/non_pullup_prob:.2f}")
        print(f"단거리 풀업이 다른 미사일보다 {pullup_prob*4/non_pullup_prob:.2f}배 더 자주 발사될 것으로 예상됩니다.")
    else:
        print("비단거리 풀업 미사일의 확률이 0이므로 비율을 계산할 수 없습니다.")


if __name__ == "__main__":
    # 전체 지역 및 미사일 목록 정의

    region_airport = [
        "공군기지1",
        "공군기지2",
        "공군기지3",
        "공군기지4",
        "공군기지5",
        "공군기지6",
        "공군기지7",
    ]

    region_missile_attack = [
        "미사일공격기지1",
        "미사일공격기지2",
        "미사일공격기지3",
        "미사일공격기지4",
        "미사일공격기지5",
        "미사일공격기지6",
        "미사일공격기지7",
        "미사일공격기지8",
    ]

    region_missile_defense = [
        "미사일방어기지1",
        "미사일방어기지2",
        "미사일방어기지3",
        "미사일방어기지4",
        "미사일방어기지5",
        "미사일방어기지6",
        "미사일방어기지7",
        "미사일방어기지8",
    ]

    region_alliance = [
        "한미연합1",
        "한미연합2",
        "한미연합3",
        "한미연합4",
        "한미연합5",
        "한미연합6(오키나와)",
        "한미연합7(괌)",
    ]

    region_commander = [
        "지휘통제1",
        "지휘통제2",
        "지휘통제3",
        "지휘통제4",
        "지휘통제5",
        "지휘통제6",
        "지휘통제7",
    ]

    region_supply = [
        "민강공항1",
        "민간공항2",
        "민간공항3",
        "민간공항4",
        "군수지원1",
        "군수지원2",
        "군수지원3",
        "군수지원4",
        "군수지원5",
        "군수지원6",
        "군수지원7",
        "군수지원8",
        "항만1",
        "항만2",
        "항만3",
        "항만4",
        "항만5",
    ]

    region_vip = [
        "주요인구시설1",
        "주요인구시설2",
        "주요인구시설3",
        "주요인구시설4",
        "주요인구시설5",
        "주요인구시설6",
    ]

    all_regions = (
        region_airport
        + region_missile_attack
        + region_missile_defense
        + region_alliance
        + region_commander+region_supply
        + region_vip
    )

    all_missiles = ["단거리 풀업", "단거리 탄도탄", "단거리 장사정","중거리 탄도탄","장거리 탄도탄"] #

    # IntentionRegionMissileMapping 설정
    mapping = IntentionRegionMissileMapping()

    # 의도는 총 7개로 이루어진다.

    # 의도 1
    mapping.add_intention_pair(
        "핵 공격 여건조성/한국의 거부적 억제수단 무력화",
        "작전적 타격수단, 미사일방어체계",
        region_airport+ region_missile_defense,
        all_missiles,
    )

    # 의도 2

    mapping.add_intention_pair(
        "핵 공격 여건조성/한국의 보복적 억제수단 무력화",
        "전략적 타격수단",
        region_missile_attack,
        all_missiles,
    )

    # 의도 3

    mapping.add_intention_pair(
        "핵 공격 여건조성/한미 연합작전 수행능력 무력화",
        "미사일방어체계, 한미연합전력",
       region_missile_defense+region_alliance,
        all_missiles,
    )

    # 의도 4

    mapping.add_intention_pair(
        "재래식 전면공격 여건조성/아군의 전쟁지도 및 지휘능력 마비",
        "전쟁지휘통제시설",
        region_commander,
        all_missiles,
    )

    # 의도 5

    mapping.add_intention_pair(
        "재래식 전면공격 여건조성/아군의 전구작전 수행능력 저하",
        "미사일방어체계",
        region_missile_defense,
        all_missiles,
    )

    # 의도 6

    mapping.add_intention_pair(
        "재래식 전면공격 여건조성/아군의 전쟁지속 및 전시증원능력 마비",
        "전쟁지휘통제시설, 전쟁지속지원능력",
        region_commander+region_supply,
        all_missiles,
    )

    # 의도 7

    mapping.add_intention_pair(
        "재래식 전면공격 여건조성/전쟁 수행의지 약화 및 사회적 혼란 조성",
        "정치/심리적표적",
        region_vip,
        all_missiles,
    )

    #####################

    # ScenarioDataGenerator 초기화
    data_generator = ScenarioDataGenerator(mapping, all_regions, all_missiles)

    # 데이터 생성을 위한 파라미터 설정
    time_steps = 5
    poisson_param_model = ("linear", (1, 10))  # 선형적으로 증가하는 포아송 파라미터

    # 데이터 생성
    # 랜덤 transition_matrix를 사용하여 데이터 생성
    scenario_data, used_transition_matrix, used_poisson_params = data_generator.generate_data(
        "재래식 전면공격 여건조성/전쟁 수행의지 약화 및 사회적 혼란 조성", "정치/심리적표적", time_steps, poisson_param_model=poisson_param_model
    )

    print("시나리오 데이터 형태:", scenario_data.shape)
    print("\n첫 번째 시간 단계의 데이터:")
    print(scenario_data[0])
    print("\n사용된 transition_matrix 형태:")
    print(used_transition_matrix.shape)
    print(used_transition_matrix)
    print("\n사용된 poisson_params:")
    print(used_poisson_params)
    active_regions = mapping.get_regions_missiles("재래식 전면공격 여건조성/전쟁 수행의지 약화 및 사회적 혼란 조성", "정치/심리적표적")['regions']
    analyze_transition_matrix(used_transition_matrix, all_regions, all_missiles, active_regions)

시나리오 데이터 형태: (5, 60, 5)

첫 번째 시간 단계의 데이터:
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

사용된 transition_matrix 형태:
(300, 300)
[[0.00041367 0.00093624 0.00133835 ... 0.00660607 0.01868097 0.0019908 ]
 [0.00231612 0.00413723 0.00026379 ... 0.01397708 0.00672513 0.019

In [2]:
mapping.get_all_mappings()

{('핵 공격 여건조성/한국의 거부적 억제수단 무력화',
  '작전적 타격수단, 미사일방어체계'): {'regions': {'공군기지1',
   '공군기지2',
   '공군기지3',
   '공군기지4',
   '공군기지5',
   '공군기지6',
   '공군기지7',
   '미사일방어기지1',
   '미사일방어기지2',
   '미사일방어기지3',
   '미사일방어기지4',
   '미사일방어기지5',
   '미사일방어기지6',
   '미사일방어기지7',
   '미사일방어기지8'}, 'missiles': {'단거리 장사정',
   '단거리 탄도탄',
   '단거리 풀업',
   '장거리 탄도탄',
   '중거리 탄도탄'}},
 ('핵 공격 여건조성/한국의 보복적 억제수단 무력화',
  '전략적 타격수단'): {'regions': {'미사일공격기지1',
   '미사일공격기지2',
   '미사일공격기지3',
   '미사일공격기지4',
   '미사일공격기지5',
   '미사일공격기지6',
   '미사일공격기지7',
   '미사일공격기지8'}, 'missiles': {'단거리 장사정',
   '단거리 탄도탄',
   '단거리 풀업',
   '장거리 탄도탄',
   '중거리 탄도탄'}},
 ('핵 공격 여건조성/한미 연합작전 수행능력 무력화',
  '미사일방어체계, 한미연합전력'): {'regions': {'미사일방어기지1',
   '미사일방어기지2',
   '미사일방어기지3',
   '미사일방어기지4',
   '미사일방어기지5',
   '미사일방어기지6',
   '미사일방어기지7',
   '미사일방어기지8',
   '한미연합1',
   '한미연합2',
   '한미연합3',
   '한미연합4',
   '한미연합5',
   '한미연합6(오키나와)',
   '한미연합7(괌)'}, 'missiles': {'단거리 장사정',
   '단거리 탄도탄',
   '단거리 풀업',
   '장거리 탄도탄',
   '중거리 탄도탄'}},
 ('재래식 전면공격 여건조성/아군의 전쟁지도 및 지휘

In [3]:
import pandas as pd
import numpy as np



def create_scenario_dataframe(
    scenario_datas, all_regions, all_missiles, mapping, intention_pair
):
    # 활성화된 지역과 미사일 가져오기
    active_regions_missiles = mapping.get_regions_missiles(*intention_pair)
    print(active_regions_missiles)
    active_regions = list(active_regions_missiles["regions"])
    active_missiles = list(active_regions_missiles["missiles"])

    # 데이터 준비
    time_steps, _, _ = scenario_datas.shape

    # 단일 인덱스 생성
    index = [
        f"{region}-{missile}" for region in all_regions for missile in all_missiles
    ]

    # 데이터프레임 생성
    df = pd.DataFrame(index=index, columns=range(1, time_steps + 1))

    # 데이터 채우기
    for t in range(time_steps):
        for region in all_regions:
            for missile in all_missiles:
                region_idx = all_regions.index(region)
                missile_idx = all_missiles.index(missile)

                if region_idx == 28: # 특정 지역 과 미사일 관계에 대해서 rule based로 마스킹
                    if missile_idx == 3:
                        value = scenario_datas[t, region_idx, missile_idx]
                    else:
                        value = 0
                elif region_idx == 29:
                    if missile_idx == 4:
                        value = scenario_datas[t, region_idx, missile_idx]
                    else:
                        value = 0
                elif missile_idx in [3, 4]:
                    value = 0
                else:
                    value = scenario_datas[t, region_idx, missile_idx]

                if value > 0:
                    df.loc[f"{region}-{missile}", t + 1] = value

    # NaN 값을 0으로 채우기
    df = df.fillna(0)

    return df


In [4]:
def dataframe_to_numpy(df):
    """
    DataFrame을 NumPy 배열로 변환합니다.
    인덱스와 컬럼이 숫자가 아닌 경우, 이를 숫자 인덱스로 변환합니다.
    """
    # 인덱스가 숫자가 아닌 경우 숫자 인덱스로 변환
    if not df.index.is_numeric():
        df = df.reset_index(drop=True)

    # 컬럼이 숫자가 아닌 경우 숫자 인덱스로 변환
    if not all(isinstance(col, (int, float)) for col in df.columns):
        df.columns = range(len(df.columns))

    return df.to_numpy()


def convert_dataframes_to_numpy(dataframes):
    """
    여러 DataFrame을 NumPy 배열로 변환합니다.
    """
    numpy_arrays = {}
    for name, df in dataframes.items():
        numpy_arrays[name] = dataframe_to_numpy(df)
    return numpy_arrays

# 데이터 생성

In [5]:
# import data_generator

N = 1 # sparse함을 예방하기 위한 반복횟수, 이를 통해 늘립니다. N이 증가할수록 덜 sparse해집니다.
poisson_param_model = ("linear", (30, 10)) # 예시로 매 시퀀스별 포아송 분포의 모수가 30에서 10으로 선형적으로 감소

# 모든 매핑 가져오기
all_mappings = mapping.get_all_mappings()

# 결과를 저장할 딕셔너리
results = {}

for intention_pair, _ in all_mappings.items():
    intention_pair_1, intention_pair_2 = intention_pair

    for time_steps in [*range(5)]: # 타임스텝 갯수 : 여기서는 5개를 생성합니다
        scenario_datas = None

        for _ in range(40):  # 각 조합과 time_steps에 대해 40개의 데이터 생성
            for i in range(N):
                scenario_data, used_transition_matrix, used_poisson_params = (
                    data_generator.generate_data(
                        intention_pair_1,
                        intention_pair_2,
                        time_steps,
                        poisson_param_model=poisson_param_model,
                    )
                )

                if i == 0:
                    scenario_datas = scenario_data
                else:
                    scenario_datas += scenario_data

            # 결과 저장
            key = (intention_pair_1, intention_pair_2, time_steps)
            if key not in results:
                results[key] = []
            results[key].append(scenario_datas)

# 결과 출력 (옵션)
for key, value in results.items():
    print(f"Intention pair: {key[0]} / {key[1]}")
    print(f"Time steps: {key[2]}")
    print(f"Number of datasets: {len(value)}")
    print("---")

Intention pair: 핵 공격 여건조성/한국의 거부적 억제수단 무력화 / 작전적 타격수단, 미사일방어체계
Time steps: 20
Number of datasets: 40
---
Intention pair: 핵 공격 여건조성/한국의 보복적 억제수단 무력화 / 전략적 타격수단
Time steps: 20
Number of datasets: 40
---
Intention pair: 핵 공격 여건조성/한미 연합작전 수행능력 무력화 / 미사일방어체계, 한미연합전력
Time steps: 20
Number of datasets: 40
---
Intention pair: 재래식 전면공격 여건조성/아군의 전쟁지도 및 지휘능력 마비 / 전쟁지휘통제시설
Time steps: 20
Number of datasets: 40
---
Intention pair: 재래식 전면공격 여건조성/아군의 전구작전 수행능력 저하 / 미사일방어체계
Time steps: 20
Number of datasets: 40
---
Intention pair: 재래식 전면공격 여건조성/아군의 전쟁지속 및 전시증원능력 마비 / 전쟁지휘통제시설, 전쟁지속지원능력
Time steps: 20
Number of datasets: 40
---
Intention pair: 재래식 전면공격 여건조성/전쟁 수행의지 약화 및 사회적 혼란 조성 / 정치/심리적표적
Time steps: 20
Number of datasets: 40
---


# 저장

In [6]:
import pandas as pd
from lxml import etree


# 후처리된 결과를 저장할 딕셔너리
processed_results = {}

# 기존의 results 딕셔너리를 순회하며 후처리
for key, value_list in results.items():
    intention_pair_1, intention_pair_2, time_steps = key
    intention_pair = (intention_pair_1, intention_pair_2)

    for i, scenario_datas in enumerate(value_list):
        # NumPy 배열로 변환
        scenario_datas = np.array(scenario_datas)

        # 후처리 진행
        df = create_scenario_dataframe(
            scenario_datas, all_regions, all_missiles, mapping, intention_pair
        )

        # 결과 저장
        processed_key = (
            f"scenario_{intention_pair[0]}_{intention_pair[1]}_{time_steps}_{i+1}"
        )
        processed_results[processed_key] = df


def save_multiple_dataframes_to_xml(dataframes, file_path):
    root = etree.Element("data")

    for df_name, df in dataframes.items():
        df_element = etree.SubElement(root, "dataframe")
        df_element.set("name", df_name)

        for index, row in df.iterrows():
            row_element = etree.SubElement(df_element, "row")
            etree.SubElement(row_element, "index").text = str(index)

            for column, value in row.items():
                column_name = (
                    f"col_{column}" if isinstance(column, int) else str(column)
                )
                etree.SubElement(row_element, column_name).text = str(value)

    tree = etree.ElementTree(root)
    tree.write(file_path, pretty_print=True, xml_declaration=True, encoding="utf-8")


# XML 파일로 저장
file_path = "./multiple_scenario_data_test.xml"
save_multiple_dataframes_to_xml(processed_results, file_path)
print(f"여러 DataFrame이 {file_path}로 저장되었습니다.")

{'regions': {'공군기지5', '미사일방어기지1', '공군기지1', '공군기지4', '공군기지7', '미사일방어기지4', '미사일방어기지8', '미사일방어기지2', '공군기지3', '공군기지2', '미사일방어기지5', '미사일방어기지6', '공군기지6', '미사일방어기지7', '미사일방어기지3'}, 'missiles': {'단거리 탄도탄', '중거리 탄도탄', '단거리 풀업', '장거리 탄도탄', '단거리 장사정'}}
{'regions': {'공군기지5', '미사일방어기지1', '공군기지1', '공군기지4', '공군기지7', '미사일방어기지4', '미사일방어기지8', '미사일방어기지2', '공군기지3', '공군기지2', '미사일방어기지5', '미사일방어기지6', '공군기지6', '미사일방어기지7', '미사일방어기지3'}, 'missiles': {'단거리 탄도탄', '중거리 탄도탄', '단거리 풀업', '장거리 탄도탄', '단거리 장사정'}}
{'regions': {'공군기지5', '미사일방어기지1', '공군기지1', '공군기지4', '공군기지7', '미사일방어기지4', '미사일방어기지8', '미사일방어기지2', '공군기지3', '공군기지2', '미사일방어기지5', '미사일방어기지6', '공군기지6', '미사일방어기지7', '미사일방어기지3'}, 'missiles': {'단거리 탄도탄', '중거리 탄도탄', '단거리 풀업', '장거리 탄도탄', '단거리 장사정'}}
{'regions': {'공군기지5', '미사일방어기지1', '공군기지1', '공군기지4', '공군기지7', '미사일방어기지4', '미사일방어기지8', '미사일방어기지2', '공군기지3', '공군기지2', '미사일방어기지5', '미사일방어기지6', '공군기지6', '미사일방어기지7', '미사일방어기지3'}, 'missiles': {'단거리 탄도탄', '중거리 탄도탄', '단거리 풀업', '장거리 탄도탄', '단거리 장사정'}}
{'regions': {'공군기지5', '미사일방어기지1', '공

# 불러오기

In [7]:
import pandas as pd
from lxml import etree


def load_xml_to_dataframes(file_path):
    tree = etree.parse(file_path)
    root = tree.getroot()

    dataframes = {}

    for df_element in root.findall("dataframe"):
        df_name = df_element.get("name")
        data = []

        for row_element in df_element.findall("row"):
            row_data = {}
            for child in row_element:
                # 'col_' 접두사 제거 및 가능한 경우 정수로 변환
                column_name = (
                    child.tag[4:] if child.tag.startswith("col_") else child.tag
                )
                if column_name.isdigit():
                    column_name = int(column_name)
                row_data[column_name] = child.text
            data.append(row_data)

        df = pd.DataFrame(data)

        if "index" in df.columns:
            df.set_index("index", inplace=True)

        # 데이터 타입을 float으로 변환 (가능한 경우)
        for col in df.columns:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass  # 숫자로 변환할 수 없는 경우 원래 타입 유지

        dataframes[df_name] = df

    return dataframes


# 사용 예시
file_path = "./multiple_scenario_data_test.xml"
loaded_dataframes = load_xml_to_dataframes(file_path)

for df_name, df in loaded_dataframes.items():
    print(f"\nDataFrame: {df_name}")
    print(df.head())
    print(df.dtypes)


DataFrame: scenario_핵 공격 여건조성/한국의 거부적 억제수단 무력화_작전적 타격수단, 미사일방어체계_20_1
                 1    2    3    4    5    6    7    8    9   10   11   12  \
index                                                                       
공군기지1-단거리 풀업   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
공군기지1-단거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
공군기지1-단거리 장사정  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
공군기지1-중거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
공군기지1-장거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                13   14   15   16   17   18   19   20  
index                                                  
공군기지1-단거리 풀업   0.0  0.0  0.0  0.0  0.0  0.0  7.0  0.0  
공군기지1-단거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
공군기지1-단거리 장사정  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
공군기지1-중거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
공군기지1-장거리 탄도탄  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

# 통계량 분석

In [8]:
import pandas as pd
import numpy as np
from collections import Counter


def analyze_dataframes(loaded_dataframes, all_mappings):
    # 총 데이터 개수
    total_dataframes = len(loaded_dataframes)

    # time_sequence 분포
    time_sequences = [len(df.columns) for df in loaded_dataframes.values()]
    time_sequence_distribution = Counter(time_sequences)

    # 매핑 종류 개수
    mapping_types = set()
    for df_name in loaded_dataframes.keys():
        parts = df_name.split("_")
        if len(parts) >= 3:
            mapping_types.add((parts[1], parts[2]))
    mapping_count = len(mapping_types)

    # 데이터 통계
    all_values = []
    for df in loaded_dataframes.values():
        all_values.extend(df.values.flatten())

    all_values = np.array(all_values)
    all_values = all_values[~np.isnan(all_values)]  # NaN 값 제거

    max_value = np.max(all_values)
    min_value = np.min(all_values)
    mean_value = np.mean(all_values)
    median_value = np.median(all_values)
    std_value = np.std(all_values)

    # 결과 출력
    print(f"총 데이터 개수: {total_dataframes}")
    print("\nTime Sequence 분포:")
    for seq, count in time_sequence_distribution.items():
        print(f"  길이 {seq}: {count}개")

    print(f"\n매핑 종류 개수: {mapping_count}")
    print("매핑 종류:")
    for mapping in mapping_types:
        print(f"  {mapping[0]} / {mapping[1]}")

    print("\n데이터 통계:")
    print(f"  최대값: {max_value}")
    print(f"  최소값: {min_value}")
    print(f"  평균값: {mean_value}")
    print(f"  중앙값: {median_value}")
    print(f"  표준편차: {std_value}")

    return {
        "total_dataframes": total_dataframes,
        "time_sequence_distribution": dict(time_sequence_distribution),
        "mapping_count": mapping_count,
        "mapping_types": list(mapping_types),
        "max_value": max_value,
        "min_value": min_value,
        "mean_value": mean_value,
        "median_value": median_value,
        "std_value": std_value,
    }


# 분석 실행
results = analyze_dataframes(loaded_dataframes, all_mappings)

# 추가적인 분석: 각 매핑 타입별 통계
print("\n각 매핑 타입별 통계:")
for mapping_type in results["mapping_types"]:
    print(f"\n매핑: {mapping_type[0]} / {mapping_type[1]}")
    relevant_dfs = [
        df
        for name, df in loaded_dataframes.items()
        if name.startswith(f"scenario_{mapping_type[0]}_{mapping_type[1]}")
    ]

    if relevant_dfs:
        all_values = np.concatenate([df.values.flatten() for df in relevant_dfs])
        all_values = all_values[~np.isnan(all_values)]  # NaN 값 제거

        print(f"  데이터 개수: {len(relevant_dfs)}")
        print(f"  최대값: {np.max(all_values)}")
        print(f"  최소값: {np.min(all_values)}")
        print(f"  평균값: {np.mean(all_values)}")
        print(f"  중앙값: {np.median(all_values)}")
        print(f"  표준편차: {np.std(all_values)}")
    else:
        print("  해당하는 데이터가 없습니다.")

총 데이터 개수: 280

Time Sequence 분포:
  길이 20: 280개

매핑 종류 개수: 7
매핑 종류:
  핵 공격 여건조성/한미 연합작전 수행능력 무력화 / 미사일방어체계, 한미연합전력
  핵 공격 여건조성/한국의 보복적 억제수단 무력화 / 전략적 타격수단
  재래식 전면공격 여건조성/아군의 전쟁지속 및 전시증원능력 마비 / 전쟁지휘통제시설, 전쟁지속지원능력
  재래식 전면공격 여건조성/아군의 전구작전 수행능력 저하 / 미사일방어체계
  핵 공격 여건조성/한국의 거부적 억제수단 무력화 / 작전적 타격수단, 미사일방어체계
  재래식 전면공격 여건조성/아군의 전쟁지도 및 지휘능력 마비 / 전쟁지휘통제시설
  재래식 전면공격 여건조성/전쟁 수행의지 약화 및 사회적 혼란 조성 / 정치/심리적표적

데이터 통계:
  최대값: 46.0
  최소값: 0.0
  평균값: 0.042570833333333336
  중앙값: 0.0
  표준편차: 0.9819885002511467

각 매핑 타입별 통계:

매핑: 핵 공격 여건조성/한미 연합작전 수행능력 무력화 / 미사일방어체계, 한미연합전력
  데이터 개수: 40
  최대값: 41.0
  최소값: 0.0
  평균값: 0.04275416666666667
  중앙값: 0.0
  표준편차: 0.983438482010596

매핑: 핵 공격 여건조성/한국의 보복적 억제수단 무력화 / 전략적 타격수단
  데이터 개수: 40
  최대값: 46.0
  최소값: 0.0
  평균값: 0.04317083333333333
  중앙값: 0.0
  표준편차: 0.9830855061909105

매핑: 재래식 전면공격 여건조성/아군의 전쟁지속 및 전시증원능력 마비 / 전쟁지휘통제시설, 전쟁지속지원능력
  데이터 개수: 40
  최대값: 44.0
  최소값: 0.0
  평균값: 0.044816666666666664
  중앙값: 0.0
  표준편차: 1.0198855163148897

매핑: 재래식 전면공격 여건조성/아군의 전구작전 수행능